<a href="https://colab.research.google.com/github/hagabbar/craft_prospect/blob/master/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from keras import Sequential
from keras.layers import LSTM, Dense, Embedding, SpatialDropout1D


Using TensorFlow backend.


In [2]:
"""
This is a script which will check for non-random subsequences 
from the quantum random number generator onboard the CubeSat. 
The code will first perform a benchmark suite of tests known 
as Diehard tests. We will then use the results from these 
tests in order to train and compare the results of a 
neural network approach (LSTMs).
"""

# Define important global variables here
tot_num_samples = 2000 # total number of training samples (50/50 split between rand and non-rand)
train_split = int(int(tot_num_samples/2) * 0.9) # use 90% for training, 10% for testing
bitstream_len = 32     # length of observation window of network
batch_size = 8         # batch size (how many samples NN sees during each iteration)

# Generate a set of pseudo random numbers
pseudo_rand_data_train = np.random.randint(low=0,high=2,size=(train_split,bitstream_len))
pseudo_rand_data_test = np.random.randint(low=0,high=2,size=(int(tot_num_samples/2) - train_split,bitstream_len))
pseudo_rand_labels_train = np.ones(train_split)
pseudo_rand_labels_test = np.ones(int(tot_num_samples/2) - train_split)

# Load in set of true random numbers
true_rand_data_train = np.loadtxt('true_random_nums.txt')[:train_split,:]
true_rand_data_test = np.loadtxt('true_random_nums.txt')[train_split:int(tot_num_samples/2),:]
true_rand_labels_train = np.zeros(train_split)
true_rand_labels_test = np.zeros(int(tot_num_samples/2) - train_split)

# Combine all data and labels into two arrays (x and y respectively)
X_train = np.concatenate((pseudo_rand_data_train,true_rand_data_train))
X_test = np.concatenate((pseudo_rand_data_test,true_rand_data_test))
Y_train = np.concatenate((pseudo_rand_labels_train,true_rand_labels_train))
Y_test = np.concatenate((pseudo_rand_labels_test,true_rand_labels_test))

# Randomly shuffle training sets
idx_shuffle = np.random.permutation(int(train_split*2))
X_train = X_train[idx_shuffle,:]
Y_train = Y_train[idx_shuffle]


OSError: ignored

In [0]:
print(X_test)

[[0. 0. 1. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 1.]
 [1. 1. 0. ... 1. 0. 1.]
 ...
 [0. 1. 0. ... 1. 0. 0.]
 [1. 1. 1. ... 1. 0. 1.]
 [0. 0. 0. ... 0. 1. 1.]]


In [0]:
# Set-up LSTM network
embed_dim = 128
lstm_out = 200
batch_size = 32

model = Sequential()
model.add(Embedding(2500, embed_dim,input_length = bitstream_len))
SpatialDropout1D(0.2)
model.add(LSTM(lstm_out, dropout_U = 0.2, dropout_W = 0.2))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss = 'mean_squared_error', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

/home/hunter/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(200, dropout=0.2, recurrent_dropout=0.2)`
  if __name__ == '__main__':


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 32, 128)           320000    
_________________________________________________________________
lstm_7 (LSTM)                (None, 200)               263200    
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 201       
Total params: 583,401
Trainable params: 583,401
Non-trainable params: 0
_________________________________________________________________
None


In [0]:
# Train LSTM network
model.fit(X_train, Y_train, batch_size =batch_size, epochs = 10,  verbose = 5)

Epoch 1/10
Epoch 2/10
Epoch 3/10
Epoch 4/10
Epoch 5/10
Epoch 6/10
Epoch 7/10
Epoch 8/10
Epoch 9/10
Epoch 10/10


In [0]:
score,acc = model.evaluate(X_test,Y_test,batch_size=batch_size,verbose=2)
print("Score: %.2f" % (score))
print("Validation Accuracy: %.2f" % (acc))

Score: 0.00
Validation Accuracy: 1.00
